In [ ]:
%%capture
!pip3 install --upgrade google-cloud-aiplatform
!pip3 install --upgrade kfp
!pip3 install --upgrade google-cloud-pipeline-components
!pip3 install scikit-learn
!pip3 install pandas

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [39]:
import google.cloud.aiplatform as aiplatform
import kfp
#from kfp import dsl
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, component, ClassificationMetrics, Metrics)
from google_cloud_pipeline_components import aiplatform as gcc_aip
from google_cloud_pipeline_components.types import artifact_types

https://kubeflow-pipelines.readthedocs.io/en/latest/source/kfp.dsl.html

https://pypi.org/project/google-cloud-aiplatform/

https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform

In [2]:
PROJECT_ID = "kubeflow-demos"  # @param {type:"string"}
PROJECT_NUMBER = "141610882258"
REGION = "us-central1"  # @param {type: "string"}
BUCKET_NAME = "test-fast"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

! gcloud config set project $PROJECT_ID

!gsutil mb -l $REGION $BUCKET_URI
!gsutil ls -al $BUCKET_URI

Updated property [core/project].
Creating gs://test-fast/...
ServiceException: 409 A Cloud Storage bucket named 'test-fast' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.
      1552  2022-03-24T21:14:19Z  gs://test-fast/aiplatform-2022-03-24-21:14:19.320-aiplatform_custom_trainer_script-0.1.tar.gz#1648156459408486  metageneration=1
      1774  2022-03-27T08:32:44Z  gs://test-fast/aiplatform-2022-03-27-11:32:43.707-aiplatform_custom_trainer_script-0.1.tar.gz#1648369964419591  metageneration=1
        60  2021-11-14T15:38:56Z  gs://test-fast/batch_test.csv#1636904336323978  metageneration=1
        52  2021-11-14T16:14:44Z  gs://test-fast/batch_test1.csv#1636906484597636  metageneration=1
                                 gs://test-fast/aiplatform-custom-training-2022-03-24-21:14:19.437/
                                 gs://test-fast/aiplatform-custom-training-2022-03-27-11:32:44.390/


In [20]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [4]:
aiplatform.init(
    # your Google Cloud Project ID or number
    # environment default used is not set
    project=PROJECT_ID,

    # the Vertex AI region you will use
    # defaults to us-central1
    location=REGION,

    # Google Cloud Storage bucket in same region as location
    # used to stage artifacts
    staging_bucket=BUCKET_URI,

    # the name of the experiment to use to track
    # logged metrics and parameters
    experiment='my-experiment',

    # description of the experiment above
    experiment_description='my experiment description'
)

https://pypi.org/project/google-cloud-aiplatform/

In [6]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

# AutoML training job
AutoML can be used to automatically train a wide variety of image model types. AutoML automates the following:

* Dataset preprocessing
* Feature Engineering
* Data feeding
* Model Architecture selection
* Hyperparameter tuning
* Training the model

In [8]:
dataset = aiplatform.ImageDataset.create(
    display_name="flowers_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

INFO:google.cloud.aiplatform.datasets.dataset:Creating ImageDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create ImageDataset backing LRO: projects/141610882258/locations/us-central1/datasets/5695729716623835136/operations/5550390291069927424
INFO:google.cloud.aiplatform.datasets.dataset:ImageDataset created. Resource name: projects/141610882258/locations/us-central1/datasets/5695729716623835136
INFO:google.cloud.aiplatform.datasets.dataset:To use this ImageDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.ImageDataset('projects/141610882258/locations/us-central1/datasets/5695729716623835136')
INFO:google.cloud.aiplatform.datasets.dataset:Importing ImageDataset data: projects/141610882258/locations/us-central1/datasets/5695729716623835136
INFO:google.cloud.aiplatform.datasets.dataset:Import ImageDataset data backing LRO: projects/141610882258/locations/us-central1/datasets/5695729716623835136/operations/2487942544457990144


DeadlineExceeded: 504 Deadline Exceeded

In [32]:
aiplatform.ImageDataset.list()

 resource name: projects/141610882258/locations/us-central1/datasets/5695729716623835136,
 resource name: projects/141610882258/locations/us-central1/datasets/1359326225419468800,
 resource name: projects/141610882258/locations/us-central1/datasets/2808038348130615296,
 resource name: projects/141610882258/locations/us-central1/datasets/8031123200345636864,
 resource name: projects/141610882258/locations/us-central1/datasets/8225138624136478720,
 resource name: projects/141610882258/locations/us-central1/datasets/58581979527905280]

In [7]:
DATASET_ID = '5695729716623835136'
DATASET_NAME = 'flowers_20220404145045'
res_id = f'projects/{PROJECT_NUMBER}/locations/us-central1/datasets/{DATASET_ID}'
print(res_id)
dataset = aiplatform.ImageDataset(res_id)
print(dataset.resource_name)

projects/141610882258/locations/us-central1/datasets/5695729716623835136
projects/141610882258/locations/us-central1/datasets/5695729716623835136


In [ ]:


job = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers_" + TIMESTAMP,
    prediction_type="classification",
    multi_label=False,
    model_type="CLOUD_LOW_LATENCY_1",
    base_model=None,
)

print(job)

model = job.run(
    dataset=dataset,
    model_display_name="flowers_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=1000,
    disable_early_stopping=False,
)

INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/60785950585782272?project=141610882258
INFO:google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/141610882258/locations/us-central1/trainingPipelines/60785950585782272 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/141610882258/locations/us-central1/trainingPipelines/60785950585782272 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/141610882258/locations/us-central1/trainingPipelines/60785950585782272 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/141610882258/locations/us-central1/trainingPipelines/60785950585782272 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.traini

In [ ]:
MODEL_ID = ''
res_id = f'projects/{PROJECT_NUMBER}/locations/us-central1/datasets/{MODEL_ID}'
print(res_id)
model = aiplatform.Model(res_id)
print(model.resource_name)
#model = aiplatform.Model('/projects/my-project/locations/us-central1/models/{MODEL_ID}')


In [ ]:
# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

endpoint = model.deploy()

In [ ]:
@component()
def create_dataset():
    
@component()
def train():
    
@component()
def evaluate():
    
@component()
def deploy():
    

In [51]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [52]:
@component(packages_to_install=["google-cloud-aiplatform"])
def create_dataset(my_dataset: Output[artifact_types.VertexDataset]):
    from google_cloud_pipeline_components.types import artifact_types
    from datetime import datetime
    
    TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
    
    import google.cloud.aiplatform as aiplatform

    aiplatform.init(
                    project='kubeflow-demos',
                    location='us-central1',
                    staging_bucket='gs://test-fast/',
                    experiment='my-experiment',
                    experiment_description='my experiment description')
    
    IMPORT_FILE = ("gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv")
    
    dataset = aiplatform.ImageDataset.create(
                                            display_name="flowers_" + TIMESTAMP,
                                            gcs_source=[IMPORT_FILE],
                                            import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification)

    print(dataset.resource_name)
    

@component(packages_to_install=["google-cloud-aiplatform"])
def train(my_dataset: Input[artifact_types.VertexDataset]):
    from google_cloud_pipeline_components.types import artifact_types

    from datetime import datetime
    TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
    
    import google.cloud.aiplatform as aiplatform

    aiplatform.init(
                    project='kubeflow-demos',
                    location='us-central1',
                    staging_bucket='gs://test-fast/',
                    experiment='my-experiment',
                    experiment_description='my experiment description')
    
    job = aiplatform.AutoMLImageTrainingJob(
                                            display_name="flowers_" + TIMESTAMP,
                                            prediction_type="classification",
                                            multi_label=False,
                                            model_type="CLOUD",
                                            base_model=None)

    print(job)

    model = job.run(
                    dataset=my_dataset,
                    model_display_name="flowers_" + TIMESTAMP,
                    training_fraction_split=0.8,
                    validation_fraction_split=0.1,
                    test_fraction_split=0.1,
                    budget_milli_node_hours=1000,
                    disable_early_stopping=False)
@component()
def evaluate():
    print("evaluate")
@component()
def deploy():
    print("deploy")

https://www.kubeflow.org/docs/components/pipelines/sdk-v2/v2-component-io/

In [53]:
@kfp.dsl.pipeline(name="train-opti" + TIMESTAMP)
def pipeline(
    project: str = PROJECT_ID,
    bucket: str = BUCKET_URI,
    baseline_accuracy: float = 70.0
):
    from google_cloud_pipeline_components.types import artifact_types

    create_dataset_task = create_dataset()
    
    train_task = train(create_dataset_task.outputs["my_dataset"])

In [54]:
from kfp.v2 import compiler

compiler.Compiler().compile(pipeline_func=pipeline, package_path="dag-"+TIMESTAMP+".json")

/Users/yarkoni/projects/workshop/venv/lib/python3.9/site-packages/kfp/v2/compiler/compiler.py:1278: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


In [55]:
# Instantiate PipelineJob object
pl = aiplatform.PipelineJob(
    display_name="test-opti",

    # Whether or not to enable caching
    # True = always cache pipeline step result
    # False = never cache pipeline step result
    # None = defer to cache option for each pipeline component in the pipeline definition
    enable_caching=False,

    # Local or GCS path to a compiled pipeline definition
    template_path="dag-"+TIMESTAMP+".json",

    # Dictionary containing input parameters for your pipeline
    parameter_values={},

    # GCS path to act as the pipeline root
    pipeline_root=BUCKET_URI,
)

# Execute pipeline in Vertex AI and monitor until completion
pl.run(
  # Email address of service account to use for the pipeline run
  # You must have iam.serviceAccounts.actAs permission on the service account to use it
  #service_account=service_account,

  # Whether this function call should be synchronous (wait for pipeline run to finish before terminating)
  # or asynchronous (return immediately)
  sync=False
)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/141610882258/locations/us-central1/pipelineJobs/train-opti20220404193257-20220404193301
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/141610882258/locations/us-central1/pipelineJobs/train-opti20220404193257-20220404193301')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/train-opti20220404193257-20220404193301?project=141610882258
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/141610882258/locations/us-central1/pipelineJobs/train-opti20220404193257-20220404193301 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/141610882258/loca

In [ ]:
jobs = aiplatform.PipelineJob.list()